In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import HBox, VBox, widgets


data = pd.read_csv("design_style_grid.csv")
styles = data["style"].tolist()
imp = data.important.to_numpy()
spectrum_names = [
    "Modernity: Traditional to Transitional to Modern",
    "Aesthetic: Minimalist to Maximalist",
    "Atmosphere: Rustic to Industrial",
    "Atmosphere2: Informal to Formal",
]

X_df = data.reindex(columns=spectrum_names)
X = X_df.to_numpy()
features = X_df.columns

pc1, pc2 = 0, 1

s_means = X.mean(axis=0)
s_std = X.std(axis=0)
X_norm = (X - 50) / s_std
cov = X_norm.T @ X_norm
eigenvalues, eigenvectors = np.linalg.eigh(cov)
eigenvalues, eigenvectors = np.flip(eigenvalues), np.flip(eigenvectors, axis=1)
# print("Explained Variance:")
# print(
#     f"Principal Component {pc1 + 1}: {100 * eigenvalues[pc1] / eigenvalues.sum():.2f}%"
# )
# print(
#     f"Principal Component {pc2 + 1}:"
#     f" {100 * eigenvalues[pc2] / eigenvalues.sum():.2f}%\n"
# )

k = pc2 + 1
X_trans = X_norm @ eigenvectors[:, :k]

# Add noise for plotting
X_trans = X_trans + np.random.normal(scale=0.1, size=X_trans.shape)

# Create a Plotly figure
fig = go.FigureWidget()

# Add scatter plot for the special point
user_point = fig.add_scatter(
    x=[0],
    y=[0],
    mode="markers",
    marker=dict(size=40, color="#008080"),
    text=["Your Style"],
    hoverinfo="text",
)

# Add scatter plot for normal points
colors = ["#a2d2ff", "#ffb5a7", "#bde0fe", "#c3bef0"]
color_sq = [colors[i % len(colors)] for i in range(len(X_trans))]
fig.add_scatter(
    x=X_trans[imp, pc1],
    y=X_trans[imp, pc2],
    mode="markers",
    # marker=dict(size=20, color="#c3bef0"),
    marker=dict(size=20, color=color_sq[: imp.sum()]),
    text=[s for i, s in enumerate(styles) if imp[i]],
    hoverinfo="text",
)
fig.add_scatter(
    x=X_trans[~imp, pc1],
    y=X_trans[~imp, pc2],
    mode="markers",
    # marker=dict(size=10, color="#bde0fe"),
    marker=dict(size=20, color=color_sq[imp.sum() :]),
    text=[s for i, s in enumerate(styles) if not imp[i]],
    hoverinfo="text",
)

# Customize layout
fig.update_layout(
    autosize=False,
    width=600,
    height=600,
    # xaxis=dict(showgrid=False),
    # yaxis=dict(showgrid=False),
    showlegend=False,
    plot_bgcolor="#f5f5f5",  # Transparent background
    paper_bgcolor="#f5f5f5",  # Transparent paper
    xaxis={"visible": False},  # Hide the x-axis
    yaxis={"visible": False},  # Hide the y-axis
    margin={"l": 0, "r": 0, "t": 0, "b": 0},  # Reduce margins
)


# Function to update the plot
def update_point(modern, minimalist, industrial, formal):
    usr_pnt = np.array([modern, minimalist, industrial, formal]).reshape([1, -1])
    usr_pnt = (usr_pnt - 50) / s_std
    usr_pnt = usr_pnt @ eigenvectors[:, :k]
    with fig.batch_update():
        fig.data[0].x = [usr_pnt[0, 0]]
        fig.data[0].y = [usr_pnt[0, 1]]


vboxes = []
desc_width = "150px"
for spec in spectrum_names:
    # Parse spectrum name
    s1 = spec.split(": ")[1].split(" to ")[0]
    s2 = spec.split(": ")[1].split(" to ")[-1]

    # Create slider
    slider = widgets.FloatSlider(
        value=50,  # Initial value
        min=0,  # Minimum value
        max=100,  # Maximum value
        step=1,
        description="",
        disabled=False,
        continuous_update=True,
        orientation="horizontal",
        readout=False,  # Hide the numeric value
        layout=widgets.Layout(width="300px"),  # Adjust the width as needed
    )

    # Add slider to visual
    vboxes.append(
        [
            widgets.Label(
                s1,
                layout=widgets.Layout(
                    width="100px", display="flex", justify_content="flex-end"
                ),
            ),
            slider,
            widgets.Label(s2),
        ]
    )


# Observe function to wrap update
def on_slider_change(change):
    update_point(*[row[1].value for row in vboxes])


# Link sliders to observe function
for row in vboxes:
    row[1].observe(on_slider_change, names="value")

# Initial call

ui = VBox([HBox(row) for row in vboxes])

# Display the interactive plot with sliders
display(fig, ui)

C:\Users\robert.robison\AppData\Local\Temp\ipykernel_9236\3631730935.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


FigureWidget({
    'data': [{'hoverinfo': 'text',
              'marker': {'color': '#008080', 'size': 40},
              'mode': 'markers',
              'text': [Your Style],
              'type': 'scatter',
              'uid': '9ef254f0-1ecb-4392-8a5d-07f5fe2d19a3',
              'x': [0],
              'y': [0]},
             {'hoverinfo': 'text',
              'marker': {'color': ['#a2d2ff', '#ffb5a7', '#bde0fe'], 'size': 20},
              'mode': 'markers',
              'text': [Modern, Traditional, Transitional],
              'type': 'scatter',
              'uid': '40d326d5-5eb3-44d1-8236-8192a84459f7',
              'x': array([-1.31411924,  2.67439303,  0.08906654]),
              'y': array([0.97300859, 1.14719593, 0.91056838])},
             {'hoverinfo': 'text',
              'marker': {'color': [#c3bef0, #a2d2ff, #ffb5a7, #bde0fe, #c3bef0,
                                   #a2d2ff, #ffb5a7, #bde0fe, #c3bef0, #a2d2ff,
                                   #ffb5a7, #bde0f